In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
def process_data(df, fit_scaler=True, scaler=None):
    df = df.copy()
    
    # Derive Hour from timestamp
    df['Hour'] = pd.to_datetime(df['timestamp'], unit='s').dt.hour

    # Select features
    feature_cols = ['Latitude', 'Longitude', 'Altitude', 'speed_kmh', 'Hour',
                    'temperature', 'humidity', 'windSpeed', 'pressure', 'operator']
    df_features = df[feature_cols].copy()
    
    # Fill missing values
    df_features.fillna(0, inplace=True)

    # Scale all features
    if fit_scaler:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_features)
    else:
        X_scaled = scaler.transform(df_features)

    return X_scaled, scaler

In [4]:
# Load training data
train_df = pd.read_csv("Train.csv")

# Log-transform the target
y = np.log1p(train_df['target'])

# Preprocess features
X, scaler = process_data(train_df)

In [5]:
y.describe()

count    34274.000000
mean        17.526507
std          0.834675
min         10.778977
25%         16.993564
50%         17.562659
75%         18.092177
max         19.417629
Name: target, dtype: float64

In [6]:
# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from xgboost import XGBRegressor

model = XGBRegressor(
    objective='reg:squarederror',  # for regression
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [15]:
# Predict and reverse log transform
val_preds_log = model.predict(X_val)
val_preds = np.expm1(val_preds_log)
y_val_actual = np.expm1(y_val)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_val_actual, val_preds))
mape = mean_absolute_percentage_error(y_val_actual, val_preds)
r2 = r2_score(y_val_actual, val_preds)

print(f"✅ Validation RMSE: {rmse:.2f}")
print(f"📉 MAPE: {mape:.4f}")
print(f"📈 R² Score: {r2:.4f}")


✅ Validation RMSE: 16889934.10
📉 MAPE: 0.2602
📈 R² Score: 0.8526


In [16]:
# Save model and scaler
model_filename = "qos_xgb_model.pkl"
with open(model_filename, "wb") as f:
    pickle.dump((model, scaler), f)

print(f"✅ Model and scaler saved to '{model_filename}'")


✅ Model and scaler saved to 'qos_xgb_model.pkl'
